In [1]:
#https://www.youtube.com/watch?v=IpVg0O5oWDw&ab_channel=JacobAmaral
%run "C:\\Users\\adamj\\Documents\\Projekty_Python\\Quant_trading\\Plotting_backtesting.ipynb"
%run "C:\\Users\\adamj\\Documents\\Projekty_Python\\Quant_trading\\Trading_simulation_functions.ipynb"

In [75]:
def generate_adx_signals(df):
    df['adx'] =  ta.trend.adx(high=df['high'] ,low=df['low'],close=df['close'], window=14).iloc[:,0]
    df['signal']= ''
    for i in range(1, len(df)):
        #long signal
        if df['adx'][i]>30 and df['adx'][i] >  df['adx'][i-1]:
            df['entries'][i] = True
        #short signal
        if df['adx'][i]<30 and df['adx'][i] <  df['adx'][i-1]:
            df['sell_entries'][i] = True
            
    # Closing  positions    
    for i in range(1, len(df['open'])-10):
        if df['entries'][i] == True:
            stop_loss_amount = df['close'][i]/500
            trailing_stop_loss_level = df['close'][i] - stop_loss_amount
            # Selling open position with trailing stop-loss
            for l in range(1, len(df) - i):
                    close_price = df['close'][i + l]
                    # Check if the current close price is below the trailing stop-loss level
                    if close_price < trailing_stop_loss_level:
                        df['exits'][i + l]= True
                        break
                    # Update the trailing stop-loss level if the close price is higher
                    trailing_stop_loss_level = max(trailing_stop_loss_level, close_price - stop_loss_amount)
                
        if df['sell_entries'][i] == True:
            stop_loss_amount = df['close'][i]/500
            trailing_stop_loss_level = df['close'][i] + stop_loss_amount
            # Selling open position with trailing stop-loss
            for l in range(1, len(df) - i):
                    close_price = df['close'][i + l]
                    # Check if the current close price is below the trailing stop-loss level
                    if close_price > trailing_stop_loss_level:
                        df['sell_exits'][i + l] = True
                        break
                    # Update the trailing stop-loss level if the close price is higher
                    trailing_stop_loss_level = min(trailing_stop_loss_level, close_price + stop_loss_amount)
   
    return df

In [76]:
#tickers = ['^GSPC', '^IXIC', 'EURUSD=X', '^GDAXI','USO']
#tickers = ['aapl', 'tsla', 'amzn']
tickers = ['^GSPC','BAC','AAPL', '^NDX', 'gdx']

In [77]:
#Initialize an empty list to store the results for each ticker
results = []
# Loop through each ticker in the list of tickers
for ticker in tickers:
    # Initialize a list to store temporary statistics for the current ticker
    temp_stats = []
    # Apply functions to calculate portfolio statistics for the current ticker
    temp_stats = calculate_portfolio_stats(
        generate_adx_signals(
            get_stock_data(symbol=ticker, start_date='2022-11-30', end_date='2023-12-30', interval='1h')
        )
    )
    # Set the name attribute of the temporary statistics to the current ticker
    temp_stats.name = ticker
    # Create a DataFrame from the temporary statistics
    df = pd.DataFrame(temp_stats)
    # Append the DataFrame to the list of results
    results.append(df)

In [78]:
final_df = pd.concat(results, axis=1)
# Apply the function to each row
final_df['average'] = final_df.apply(calculate_average, axis=1)
final_df

,^GSPC,BAC,AAPL,^NDX,gdx,average
Start,2022-11-30 09:30:00-05:00,2022-11-30 09:30:00-05:00,2022-11-30 09:30:00-05:00,2022-11-30 09:30:00-05:00,2022-11-30 09:30:00-05:00,NaN
End,2023-12-29 15:30:00-05:00,2023-12-29 15:30:00-05:00,2023-12-29 15:30:00-05:00,2023-12-29 15:30:00-05:00,2023-12-29 15:30:00-05:00,NaN
Period,1896,1896,1896,1896,1896,1896.000000
Start Value,10000.0,10000.0,10000.0,10000.0,10000.0,10000.000000
End Value,6140.632344,5254.85822,3612.329173,5725.101596,6006.904952,5347.965257
Total Return [%],-38.593677,-47.451418,-63.876708,-42.748984,-39.93095,-46.520347
Benchmark Return [%],20.986263,-7.11921,36.05654,45.898842,9.922045,21.148896
Max Gross Exposure [%],100.0,100.0,100.0,100.0,100.0,100.000000
Total Fees Paid,3949.258699,4402.742046,3519.501993,4254.137318,4025.111874,4030.150386
Max Drawdown [%],39.479496,49.870576,65.252151,44.895977,45.047535,48.909147
